# WallStreetBets Analysis

In [30]:
import pandas as pd
import numpy as np
import fasttext
import fasttext.util
from sqlalchemy import create_engine
from tqdm import tqdm

In [3]:
ft = fasttext.load_model('../../fastText/cc.en.300.bin')

In [4]:
fasttext.util.reduce_model(ft, 50)

In [16]:
def lookup_sentence_embedding(text):
    tokens = text.split(' ')
    word_vectors = np.array([ft.get_word_vector(t) for t in tokens])
    return word_vectors.mean(axis=0)

In [26]:
def create_comment_vectors(chunk):
    
    cleaned_chunk = pd.DataFrame({
        'date' : pd.to_datetime(chunk['created_utc'], unit='s', origin='unix').dt.date,
        'body' : chunk['body'],
        'embeddings' : chunk['body'].apply(lookup_sentence_embedding)})
    return cleaned_chunk

In [31]:
chunksize=10**4
DATABASE_URI = 'sqlite:///ft_database.db'
db = create_engine(DATABASE_URI)

wsb = pd.read_json('/home/aaruran/Documents/Git/wsbData.json', lines=True, chunksize=chunksize)

for chunk in tqdm(wsb):
    cleaned_chunk = create_comment_vectors(chunk)
    cleaned_chunk.to_sql('wsb', db, if_exists='append', index=False)

298it [05:06,  1.03s/it]
